In [ ]:
from obspy               import *
from obspy.clients        import fdsn
from matplotlib.colorbar import ColorbarBase
from matplotlib.colors   import Normalize
from obspy.geodetics.base       import gps2dist_azimuth
from obspy.geodetics import locations2degrees
from obspy.clients.fdsn    import Client, RoutingClient
from obspy.core.util.obspy_types import CustomComplex
from obspy.signal.rotate import rotate2zne
from pandas import date_range
from datetime import datetime

import os
import sys
import obspy.signal.array_analysis as AA
import matplotlib.pyplot           as plt
import matplotlib.cm               as cm
import numpy                       as np
import scipy                       as sp
import scipy.odr                   as odr
import math
import obspy
import timeit

import warnings
warnings.filterwarnings('ignore')

In [ ]:
## start timer for runtime
start_timer = timeit.default_timer()

In [ ]:
## generate configuration object
config = {}

config['fdsn_client'] = Client('BGR')
# config['fdsn_client'] = RoutingClient('eida-routing')


#starttime for data process
config['starttime'] = UTCDateTime("2019-06-01T00:00")
config['days'] = 0
config['dates'] = date_range(start=datetime.strptime(str(config['starttime']), '%Y-%m-%dT%H:%M:%S.%fZ'), end=datetime.strptime(str(config['starttime']+config['days']*86400), '%Y-%m-%dT%H:%M:%S.%fZ'))

# config['save_path'] = "/home/andbro/kilauea-data/LNM/data/GRF/"
config['save_path'] = "/export/data/LNM/data/GRF/tmp/"

#PFO array information
config['network'] = {'GRA1':'GR','GRA2':'GR','GRA3':'GR','GRA4':'GR','GRB1':'GR','GRB2':'GR','GRB3':'GR','GRB4':'GR','GRB5':'GR','GRC1':'GR','GRC2':'GR','GRC3':'GR','GRC4':'GR'}
config['array_stations'] = ['GRA1','GRA2','GRA3','GRA4','GRB1','GRB2','GRB3','GRB4','GRB5','GRC1','GRC2','GRC3','GRC4']
# config['misorientations'] =  [0.,0.,0.,0.,0.,0.,0.]
config['misorientations'] =  np.arange(0, len(config['array_stations']))

config['array_station'] =[]
config['misorientation'] =[]

# config['subarray_mask'] = [0,2,4,6,8,10,12] # random
config['subarray_mask'] = [0,1,2,3] # only A
# config['subarray_mask'] = [4,5,6,7,8] # only B
# config['subarray_mask'] = [9,10,11,12] # only C
# config['subarray_mask'] = [5,6,7,8,9,10,11,12] # B and C

config['subarray'] = np.arange(len(config['array_stations']))

# config['subarray_stations'] = [config['array_stations'][i] for i in config['subarray']]
config['subarray_misorientation'] = [config['misorientations'][i] for i in config['subarray']]
config['remove_misorientations'] = False
config['samples'] = 86400*20

#parameter for array-derivation
config['filter_type'] = 'highpass'
# config['prefilt'] = (0.005, 0.01, 5, 8)
# config['freq1'] = 0.014   #0.014 for Spudich    and  0.073 for Langston
config['freq1'] = 0.01
config['freq2'] = 5.0


# adr parameters -> not required for rotations only (but for strain)
config['vp'] = 5400  #6264. #1700
config['vs'] = 3000  #3751. #1000

config['vp'] = 5000
config['vs'] = 5000/1.7
config['sigmau'] = 1e-10

In [ ]:
# ## generate configuration object
# config = {}

# config['fdsn_client'] = Client(base_url="http://jane", timeout=200)
# config['fdsn_client2'] = Client("LMU")

# config['year'] = 2019
# config['project'] = "all"

# #starttime for data process
# config['starttime'] = UTCDateTime(f"{config['year']}-06-01T00:00")
# config['days'] = 1
# config['dates'] = date_range(start=datetime.strptime(str(config['starttime']), '%Y-%m-%dT%H:%M:%S.%fZ'), end=datetime.strptime(str(config['starttime']+config['days']*86400), '%Y-%m-%dT%H:%M:%S.%fZ'))

# # config['save_path'] = "/home/andbro/kilauea-data/LNM/data/GRF/"
# config['save_path'] = f"/export/data/LNM/data/RMY/{config['project']}/"

# config['logfile'] = f"{config['project']}.log"

# #PFO array information
# config['network'] = {'FUR':'GR', 'FFB1':'BW','FFB2':'BW','FFB3':'BW','GELB':'BW','TON':'BW','ALFT':'BW','BIB':'BW','GRMB':'BW'}
# config['array_stations'] = ['FUR','FFB1','FFB2','FFB3','GELB','GRMB','TON','ALFT','BIB']

# ## define subarrays
# config['subarray_mask'] = [0,1,2,3,4,5,6,7,8] # all
# #config['subarray_mask'] = [0,1,2,3] # only inner


# config['subarray'] = np.arange(len(config['array_stations']))

# # config['subarray_stations'] = [config['array_stations'][i] for i in config['subarray']]

# config['remove_misorientations'] = False
# # config['misorientations'] =  [0.,0.,0.,0.,0.,0.,0.]
# config['misorientations'] =  np.arange(0, len(config['array_stations']))
# config['subarray_misorientation'] = [config['misorientations'][i] for i in config['subarray']]


# config['samples'] = 86400*20

# #parameter for array-derivation
# config['filter_type'] = "highpass"
# # config['prefilt'] = (0.001, 0.01, 5, 10)
# config['freq1'] = 0.014   #0.014 for Spudich    and  0.073 for Langston
# config['freq2'] = 1.5


# # adr parameters -> not required for rotations only (but for strain)
# config['vp'] = 1000 #6264. #1700
# config['vs'] = 560  #3751. #1000
# config['sigmau'] = 1e-7

In [ ]:
def __get_inventory_and_distances(config):
    coo = []
    for i, station in enumerate(config['subarray_stations']):

        if station == "FUR":
            net = "GR"
        else:
            net = config['network']

        inven = config['fdsn_client'].get_stations(network=net,
                                                   station=station,
                                                   channel='BHZ',
                                                   starttime=config['starttime'],
                                                   endtime=config['starttime']+86400,
                                                   level='response'
                                                  )
        l_lon =  float(inven.get_coordinates('%s.%s..BHZ'%(net,station))['longitude'])
        l_lat =  float(inven.get_coordinates('%s.%s..BHZ'%(net,station))['latitude'])
        height = float(inven.get_coordinates('%s.%s..BHZ'%(net,station))['elevation'])
        if i == 0:
            o_lon, o_lat, o_height = l_lon, l_lat, height

        lon, lat = obspy.signal.util.util_geo_km(o_lon,o_lat,l_lon,l_lat)
        coo.append([lon*1000,lat*1000,height-o_height])  #convert unit from km to m

    return np.array(coo)

In [ ]:
def __check_samples_in_stream(st, config):

    for tr in st:
        if tr.stats.npts != config['samples']:
            print(f" -> removing {tr.stats.station} due to improper number of samples ({tr.stats.npts} not {config['samples']})")
            st.remove(tr)

    return st

In [ ]:
def __get_data(config):


    tbeg=config['starttime'];
    tend=tbeg+86400

    config['subarray'] = []
    config['code'] = {}
    config['merged'] = False
    tsz, tsn, tse = [],[],[]
    for k, station in enumerate(config['array_stations']):

        store = True
        if k in config['subarray_mask']:

            print(' requesting '+config['network'][station]+'.'+station+'..BH*')

            try:
                try:
                    stats = config['fdsn_client'].get_waveforms(network=config['network'][station],
                                                      station=station,
                                                      location='',
                                                      channel='BH*',
                                                      starttime=tbeg-1,
                                                      endtime=tend+1,
                                                      # attach_response=True,
                                                     )
                    config['code'][station] = "B"
                except:
                    stats = config['fdsn_client'].get_waveforms(network=config['network'][station],
                                                      station=station,
                                                      location='',
                                                      channel='HH*',
                                                      starttime=tbeg-1,
                                                      endtime=tend+1,
                                                      # attach_response=True,
                                                     )
                    config['code'][station] = "H"

            except Exception as E:
                print(E)
                print(" -> get_waveforms() failed...")
                continue

            try:
                inv = config['fdsn_client'].get_stations(network=config['network'][station],
                                                         station=station,
                                                         channel=f"{config['code'][station]}H*",
                                                         starttime=tbeg-1,
                                                         endtime=tend+1,
                                                         level='response'
                                                         )
            except Exception as E:
                print(E)
                print(" -> get_stations() failed...")
                continue

            if len(stats) > 3:
                print(f" -> merging stream. Length: {len(stats)} -> 3")
                config['merged'] = True
                stats.merge(method=1,fill_value="latest")


            # stats.remove_response(inventory=inv,
            #                       output="VEL",
            #                       # pre_filt=config['prefilt'],
            #                       # taper=False,
            #                       zero_mean=True,
            #                       )
            stats.remove_sensitivity(inv)

            stats.sort()
            stats.reverse()

            #correct mis-alignment
            if station[-1] in ['1','2','3']:
                print(" -> applying rotation ZNE")
                stats.rotate(method="->ZNE",inventory=inv)

            if config['remove_misorientations']:
                stats[0].data, stats[1].data, stats[2].data = rotate2zne(stats[0],0,-90,stats[1],config['subarray_misorientation'][config['subarray_stations'].index(station)],0, stats[2],90+config['subarray_misorientation'][config['subarray_stations'].index(station)],0)

            ## new by AB
            stats.resample(sampling_rate=20)
            stats.detrend('linear')
            stats.detrend('simple')

            if config['filter_type'] == "bandpass":
                stats.filter('bandpass',freqmin=config['freq1'],freqmax=config['freq2'],corners=4,zerophase=True)
            elif config['filter_type'] == "highpass":
                stats.filter('highpass', freq=config['freq1'], corners=4, zerophase=True)


            stats.trim(tbeg,tend,nearest_sample=False)


            skip = False
            for n, tr in enumerate(stats):
                if tr.stats.npts != config['samples']:
                    if tr.stats.npts == config['samples']+1:
                        tr.data = tr.data[:-1]
                        print(" -> reducing NPTS by one!") if n == 1 else None
                    else:
                        print(f" -> neglecting {tr.stats.station} due to improper number of samples ({tr.stats.npts} not {config['samples']})") if n == 1 else None
                        skip = True

            if not skip:
                try:
                    tsz.append(stats.select(station=station, channel="*Z")[0].data)
                    tsn.append(stats.select(station=station, channel="*N")[0].data)
                    tse.append(stats.select(station=station, channel="*E")[0].data)
                    config['subarray'].append(k)
                except:
                    print(" -> stream data could not be appended!")

#    print(np.shape(tsz),np.shape(tsn),np.shape(tse))
    print(f" Array data retrival for {len(config['subarray'])} of {len(config['subarray_mask'])} stations completed!")

    return np.array(tse), np.array(tsn), np.array(tsz), config

In [ ]:
def __compute_ADR(tse, tsn, tsz, config, st_template):

    print(' ADR is executing...')

    try:
        result = AA.array_rotation_strain(np.arange(len(config['subarray'])),
                                          np.transpose(tse),
                                          np.transpose(tsn),
                                          np.transpose(tsz),
                                          config['vp'],
                                          config['vs'],
                                          config['distances'],
                                          config['sigmau'],
                                      )
    except Exception as E:
        print(E)
        print("\n -> failed to compute ADR...")
#        sys.exit()

    #(Rotation trace)
    rotsa = st_template.copy()
    rotsa[0].data = result['ts_w3']
    rotsa[1].data = result['ts_w2']
    rotsa[2].data = result['ts_w1']
    
    rotsa[0].stats.channel='BJZ'
    rotsa[1].stats.channel='BJN'
    rotsa[2].stats.channel='BJE'
    
    for tr in rotsa:
        tr.stats.station = 'RGRF'
        tr.stats.network = 'GR'

    rotsa.detrend("simple")

#     gradient_ZNE = result['ts_ptilde'] #u1,1 u1,2 u1,3 u2,1 u2,2 u2,3
#     u_ee=gradient_ZNE[:,0]
#     u_en=gradient_ZNE[:,1]
#     u_ez=gradient_ZNE[:,2]
#     u_ne=gradient_ZNE[:,3]
#     u_nn=gradient_ZNE[:,4]
#     u_nz=gradient_ZNE[:,5]


    #(Gradient trace)
    #      Gradient = o_stats.copy()        #information of the central station
    #      Gradient.append(o_stats[0].copy())
    #      Gradient.append(o_stats[0].copy())
    #      Gradient.append(o_stats[0].copy())
    #      Gradient[0].data = u_ee
    #      Gradient[1].data = u_en
    #      Gradient[2].data = u_ez
    #      Gradient[3].data = u_ne
    #      Gradient[4].data = u_nn
    #      Gradient[5].data = u_nz
    #      Gradient[0].stats.channel='uee'
    #      Gradient[1].stats.channel='uen'
    #      Gradient[2].stats.channel='uez'
    #      Gradient[3].stats.channel='une'
    #      Gradient[4].stats.channel='unn'
    #      Gradient[5].stats.channel='unz'

    return rotsa

In [ ]:
def __get_distances(config):

    cc_lon, cc_lat,cc_height = 0,0,0
    for c in config['coordinates']:
        cc_lon += c[0]
        cc_lat += c[1]
        cc_height += c[2]

    cc_lon /= len(config['coordinates'])
    cc_lat /= len(config['coordinates'])
    cc_height /= len(config['coordinates'])

    ## coordinates of GRC4
#     cc_lon, cc_lat, cc_height = 11.52495, 49.0857, 503.  
    
    dists = []
    for lon,lat,height in config['coordinates']:
        dist_x, dist_y = obspy.signal.util.util_geo_km(cc_lon, cc_lat, lon, lat)
        dists.append([dist_x*1000,dist_y*1000, height-cc_height])  #convert unit from km to m

    config['centroid'] = [cc_lon, cc_lat, cc_height]
    config['distances'] = np.array(dists)
    return config

In [ ]:
def __get_inventory(config):
    coords = []
    for i, station in enumerate(config['subarray_stations']):
        try:
            inven = config['fdsn_client'].get_stations(network=config['network'][station],
                                                       station=station,
                                                       channel=f"{config['code'][station]}H*",
                                                       starttime=config['starttime'],
                                                       endtime=config['starttime']+86400,
                                                       level='response'
                                                      )
        except:
            print("next")
        l_lon =  float(inven.get_coordinates(f"{config['network'][station]}.{station}..{config['code'][station]}HZ")['longitude'])
        l_lat =  float(inven.get_coordinates(f"{config['network'][station]}.{station}..{config['code'][station]}HZ")['latitude'])
        height = float(inven.get_coordinates(f"{config['network'][station]}.{station}..{config['code'][station]}HZ")['elevation'])

        # print(inven.get_orientation(f"GR.{station}..BHZ", config['starttime']))

        coords.append([l_lon,l_lat, height])
    config['coordinates'] = np.array(coords)
    return config
    coords = []
    for i, station in enumerate(config['subarray_stations']):
        try:
            inven = config['fdsn_client'].get_stations(network=config['network'][i],
                                                       station=station,
                                                       channel='BHZ',
                                                       starttime=config['starttime'],
                                                       endtime=config['starttime']+86400,
                                                       level='response'
                                                      )
        except:
            continue

        l_lon =  float(inven.get_coordinates('%s.%s..BHZ'%(config['network'][i],station))['longitude'])
        l_lat =  float(inven.get_coordinates('%s.%s..BHZ'%(config['network'][i],station))['latitude'])
        height = float(inven.get_coordinates('%s.%s..BHZ'%(config['network'][i],station))['elevation'])


        coords.append([l_lon,l_lat, height])
    config['coordinates'] = np.array(coords)
    return config

In [ ]:
def __create_stream(config):
    
    from obspy import Stream
    from numpy import zeros
    from obspy.core.util import AttribDict

    st = Stream
    def __trace(config):
        tr = Trace()
        tr.stats.starttime = config['starttime']
        tr.stats.npts = config['samples']
        tr.stats.sampling_rate = int(config['samples']/86400)
    #     tr.data = zeros(config['samples'])

        tr.stats.coordinates = AttribDict({'latitude':  config['centroid'][1],
                                           'longitude': config['centroid'][0],
                                           'elevation': config['centroid'][2],
                                           'local_depth': 0.0,
                                          })
        return tr

    st = st(traces=[__trace(config),__trace(config),__trace(config)])
    
    return st 

In [ ]:
#### MAIN ##########################


## loop
for date in config['dates']:

    config['subarray_stations'] = [config['array_stations'][i] for i in config['subarray_mask']]


    start_timer1 = timeit.default_timer()
    print(f"\n ____________________________________ \n Working on {date}")

    ## update current date
    config['starttime'] = UTCDateTime(date)

    ## request data
    tse, tsn, tsz, config = __get_data(config)

    ## test if enough stations for ADR are available otherwise continue
    if tse.shape[0] <3 or  tsn.shape[0] <3 or  tsz.shape[0] <3:
        print(" -> not enough stations (< 3) for ADR computation!")
        continue


    ## update stations in subarray
    config['subarray_stations'] = [config['array_stations'][i] for i in config['subarray']]
#     config['network'] = [config['network'][i] for i in config['subarray']]

    ## get inventory and coordinates/distances
#     config['distances'] = __get_inventory_and_distances(config)
    
    config = __get_inventory(config)
    config = __get_distances(config)

    ## create a stream template at centroid location
    st_template = __create_stream(config)
    
    ## compute array derived rotation
    rot = __compute_ADR(tse, tsn, tsz, config, st_template)


    date_str = str(config['starttime'].date).replace("-","")
    config['filename'] = date_str[:6]

    #check if directory is nonexist, then creat
    if os.path.exists(config['save_path']+config['filename']):
        print(' directory %s exists'%(config['save_path']+config['filename']))
    else:
        print(' directory %s is creating...'%(config['save_path']+config['filename']))
        os.mkdir(config['save_path']+config['filename'])
    #    os.touch(config['save_path']+config['filename']+"log.txt")



    print(' data is written ...')
    rot.select(channel="*Z").write(config['save_path']+config['filename']+'/rot_Z_%s.mseed'%(date_str),format='MSEED', encoding="FLOAT64")
    rot.select(channel="*N").write(config['save_path']+config['filename']+'/rot_N_%s.mseed'%(date_str),format='MSEED', encoding="FLOAT64")
    rot.select(channel="*E").write(config['save_path']+config['filename']+'/rot_E_%s.mseed'%(date_str),format='MSEED', encoding="FLOAT64")

#     stats.select(channel="*Z").write(config['save_path']+config['filename']+'/tra_Z_%s.mseed'%(date_str),format='MSEED', encoding="FLOAT64")
#     stats.select(channel="*N").write(config['save_path']+config['filename']+'/tra_N_%s.mseed'%(date_str),format='MSEED', encoding="FLOAT64")
#     stats.select(channel="*E").write(config['save_path']+config['filename']+'/tra_E_%s.mseed'%(date_str),format='MSEED', encoding="FLOAT64")

    
    stop_timer1 = timeit.default_timer()
    print(f"\n Runtime: {round((stop_timer1 - start_timer1)/60,2)} minutes")
